In [2]:
import torch, sys
sys.path.insert(0, '../')
from my_utils import gpu_utils
import importlib, gc
from my_utils.alignment_features import *
import my_utils.alignment_features as afeatures
importlib.reload(afeatures)
import gnn_utils.graph_utils as gutils

In [3]:
# !pip install torch-geometric
# !pip install tensorboardX

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

#  print(torch.version.cuda)
#  print(torch.__version__)    

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt




In [5]:
from my_utils import align_utils as autils, utils
import argparse
from multiprocessing import Pool
import random

# set random seed
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)

params = argparse.Namespace()


params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-grc-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses = list(pros.keys())
params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-heb-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses.extend(list(pros.keys()))
all_verses = list(set(all_verses))
print(len(all_verses))

params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi_lang_list.txt"
editions, langs = autils.load_simalign_editions(params.editions_file)
current_editions = [editions[lang] for lang in langs]

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    
    verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_inter, verse_aligns_gdfa
    

verse_alignments_inter = {}
verse_alignments_gdfa = {}
args = []
for i,verse in enumerate(all_verses):
    args.append((verse, current_editions[:]))

#print('going to get alignments')
#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(all_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
    #verse_alignments_inter[verse] = verse_aligns_inter
    #verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
#torch.save(verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter.pickle")
#torch.save(verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')


#print('reading inter verse alignments')
#verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#for verse in all_verses[:]:
#    if len(verse_alignments_inter[verse].keys()) < 10:
#        all_verses.remove(verse)

24159


In [6]:
EPS = 1e-15

def _diag(x):
    eye = torch.eye(x.size(0)).type_as(x)
    out = eye * x.unsqueeze(1).expand(x.size(0), x.size(0))
    return out

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, edge_features, n_cluster=32):
        super(Decoder, self).__init__()
        #self.feature_encoder = afeatures.FeatureEncoding(edge_features)
        self.features_size = sum([x.out_dim for x in edge_features])
        self.representataion_size = (input_size - self.features_size)

        self.transfer = nn.Sequential(nn.Linear(input_size, hidden_size*2), nn.ReLU(), nn.Dropout(drop_out),
                            #nn.Linear(hidden_size*2, hidden_size), nn.ReLU(), nn.Dropout(drop_out),
                            nn.Linear(hidden_size*2, 1))

        #self.transfer = nn.Sequential(nn.ELU(), nn.Linear(n_cluster*2, 1), nn.ELU())

        #self.n_cluster = n_cluster                
        #self.cluster = nn.Sequential(nn.Linear(int((input_size - len(edge_features))/2), hidden_size*2), nn.ELU(), nn.Linear(hidden_size*2, 2*n_cluster))
        #self.actual_cluster = nn.Linear(2*n_cluster, n_cluster)
        #self.cos = nn.CosineSimilarity(dim=1)        
        #self.dist = nn.PairwiseDistance()
        #self.gnn_transform = nn.Sequential(nn.Linear(self.representataion_size, hidden_size), nn.ReLU(), nn.Dropout(drop_out))
        self.counter = 0

        self.objective = 'link_prediction'
    def forward(self, z, edge_index, sigmoid = True):
        if self.features_size > 0:
            if self.objective == 'link_prediction':
                edge_index_np = edge_index.cpu().numpy()
                val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
                val_indices = np.squeeze(np.asarray(val_indices))
                vals = x_edge_vals2[val_indices, :]
            elif self.objective == 'sequence_prediction':
                vals = torch.zeros((edge_index.shape[1], self.features_size)).to(dev)


            features = self.feature_encoder(vals.to(dev), dev)
            #features = vals.to(dev)
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            self.counter += 1

            #rep = self.gnn_transform(torch.cat((h1, h2), dim=1))
            res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2), features), dim=1))
            #res = self.transfer(features)
        else:
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            
            res = self.transfer(torch.cat((h1, h2), dim=-1))

            #res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2)), dim=1))
            #res = torch.sum(torch.pow(F.softmax(self.cluster(h1)/1, dim=1) - F.softmax(self.cluster(h2)/1, dim=1), 2), dim=1)
            #res = self.cos(self.cluster(h1), self.cluster(h2))
            #res = - self.dist(self.cluster(h1), self.cluster(h2))
            #print(res)
        res = torch.sigmoid(res) if sigmoid else res
        return res

    def set_objective(self, objective):
        self.objective = objective
        
    def clustering_loss(self, z, nodes, adjacency):
        s = self.actual_cluster(torch.relu(self.cluster(z[nodes])))
        s = torch.softmax(s, dim=-1)
        entropy_loss = (-s * torch.log(s + EPS)).sum(dim=-1).mean()

        ss = torch.matmul(s.transpose(0, 1), s)
        i_s = torch.eye(self.n_cluster).type_as(ss)
        ortho_loss = torch.norm(
            ss / torch.norm(ss, dim=(-1, -2), keepdim=True) -
            i_s / torch.norm(i_s), dim=(-1, -2))
        ortho_loss = torch.mean(ortho_loss)

        adjacency = adjacency.to(dev).float()
        out_adj = torch.matmul(s.transpose(0, 1),torch.sparse.mm(adjacency, s))
        # MinCUT regularization.
        mincut_num = torch.trace(out_adj)
        #d_flat = torch.einsum('ij->i', adjacency) # FIXME since I don't consider the whole adjacency matrix this could be a source of problem
        d_flat = torch.sparse.sum(adjacency, dim=1).to_dense()
        d = _diag(d_flat)
        mincut_den = torch.trace(
            torch.matmul(torch.matmul(s.transpose(0, 1), d), s))
        mincut_loss = -(mincut_num / mincut_den)
        mincut_loss = torch.mean(mincut_loss)

        return ortho_loss, mincut_loss, entropy_loss
    
    def get_alignments(self, z, edge_index):
        h1 = z[edge_index[0, :]]
        h2 = z[edge_index[1, :]]
        
        h1 = torch.softmax(self.cluster(h1), dim=1)
        h2 = torch.softmax(self.cluster(h2), dim=1)

        h1_max = torch.argmax(h1, dim=1)
        h2_max = torch.argmax(h2, dim=1)

        h1_cluster = torch.zeros(*h1.shape)
        h2_cluster = torch.zeros(*h2.shape)

        h1_cluster[range(h1.size(0)), h1_max] = 1
        h2_cluster[range(h2.size(0)), h2_max] = 1
        res = torch.max(h1_cluster * h2_cluster, dim=1).values

        #res = h1 * h2
        #res = torch.sum(res, dim = 1)
        return torch.unsqueeze(res, dim=1)

In [7]:
importlib.reload(afeatures)

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, features, n_head = 2, edge_feature_dim = 0,):
        super(Encoder, self).__init__()
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels, heads= n_head)
        self.conv2 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= 1)
        self.fin_lin = nn.Linear(out_channels, out_channels)
        

        self.feature_encoder = afeatures.FeatureEncoding(features, [normalized_tag_frequencies, word_vectors])

    def forward(self, x, edge_index):
        x = self.feature_encoder(x, dev)
        x = F.elu(self.conv1(x, edge_index, ))
        x = F.elu(self.conv2(x, edge_index))
        return F.relu(self.fin_lin(x))

In [8]:
def freeze_encoders_embedding(encoder):
    for i,ft in enumerate(encoder.feature_types):
        if ft.type == MAPPING:
            print('doing it')
            encoder.layers[i] = afeatures.MappingEncoding(encoder.layers[i].emb.weight, freeze=True)

In [9]:
def clean_memory():
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

class DataEncoder():

    def __init__(self, data_loader, model):
        self.data_loader = data_loader
        self.model = model
    
    def __iter__(self):
        for i,batch in enumerate(tqdm(self.data_loader)):
            
            x = batch['x'][0].to(dev)
            edge_index = batch['edge_index'][0].to(dev)
            verse = batch['verse'][0]

            if verse in masked_verses:
                continue

            try:
                if mask_language:
                    x[:, 0] = 0
                z = self.model.encode(x, edge_index)
                
            except Exception as e:
                global sag, khar, gav
                sag, khar, gav =  (i, batch, verse)
                print(e)
                1/0
            
            yield z, verse, i, batch

def train(epoch, data_loader, max_batches=999999999):
    global optimizer
    total_loss = 0
    model.train()

    data_encoder = DataEncoder(data_loader, model)

    for z, verse, i, batch in data_encoder:
        optimizer.zero_grad()
        
        target = batch['pos_classes'][0].to(dev)
        _, labels = torch.max(target, 1)
        
        index = batch['pos_index'][0].to(dev)
        preds = model.decoder(z, index)

        loss = criterion(preds, labels)
        loss = loss * target.shape[0]

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if i % 500 == 499:
            print(f"loss: {total_loss}")
            total_loss = 0
            test(epoch, test_data_loader)
            model.train()

        if i == max_batches:
            break
    
    print(f"total train loss: {total_loss}")


In [10]:
class POSDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, n_class, drop_out=0):
        super(POSDecoder, self).__init__()

        self.transfer = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Dropout(drop_out),
                        nn.Linear(hidden_size, n_class))

    def forward(self, z, index):
        h = z[index, :]

        res = self.transfer(h)

        return res

In [64]:
def test(epoch, testloader, filter_wordtypes=None):
    print('testing',  epoch)
    model.eval()
    total = 0
    correct = 0

    data_encoder = DataEncoder(testloader, model)
    
    with torch.no_grad():
        for z, verse, i, batch in data_encoder:
            
            target = batch['pos_classes'][0].to(dev)
            index = batch['pos_index'][0].to(dev)
            
            if filter_wordtypes != None:
                non_filtered_words = filter_wordtypes[batch['x'][0][:, 9].long()] == 1
                non_filtered_words = non_filtered_words[index]
                index = index[non_filtered_words]

                target = target[non_filtered_words, :]

            preds = model.decoder(z, index)
            
            if preds.size(0) > 0:
                _, predicted = torch.max(preds, 1)
                _, labels = torch.max(target, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'test, epoch: {epoch}, total:{total} ACC: {correct/total}')
    clean_memory()


In [78]:
print(wordtype_frequencies[736])
print(wordtype_frequencies[1473])
print(wordtype_frequencies[3683])
print(wordtype_frequencies[7367])
print(wordtype_frequencies[14733])

tensor(82.)
tensor(30.0000)
tensor(8.0000)
tensor(3.0000)
tensor(1.0000)


In [83]:
frequent_words = torch.zeros(word_frequencies.size(0))
frequent_words[word_frequencies > -1] = 1

test(1, test_data_loader, filter_wordtypes=frequent_words)

  2%|▏         | 5/250 [00:00<00:05, 44.24it/s]

testing 1


100%|██████████| 250/250 [00:03<00:00, 65.10it/s]

test, epoch: 1, total:7477 ACC: 0.7482947706299318


In [13]:
import pickle

train_verses = all_verses[:]
test_verses = all_verses[:] 
editf1 = 'fin-x-bible-helfi'
editf2 = "heb-x-bible-helfi"


if 'jpn-x-bible-newworld' in  current_editions[:]:
     current_editions.remove('jpn-x-bible-newworld')
if 'grc-x-bible-unaccented' in  current_editions[:]:
     current_editions.remove('grc-x-bible-unaccented')

# train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word.pickle", map_location=torch.device('cpu'))
# #train_dataset, train_nodes_map = create_dataset(train_verses, verse_alignments_inter, small_editions)
# features = train_dataset.features
# train_nodes_map = train_dataset.nodes_map
# #edge_index_intra_sent = train_dataset.edge_index_intra_sent
# #test_edge_index_intra_sent = edge_index_intra_sent

# # test_dataset, test_nodes_map = create_dataset(test_verses, verse_alignments_inter, small_editions)
# test_dataset, test_nodes_map = train_dataset, train_nodes_map
# test_verses = train_verses
# print(train_dataset.x.shape)

# # gutils.augment_features(test_dataset)
# # x_edge, features_edge = gutils.create_edge_attribs(train_nodes_map, train_verses, small_editions, verse_alignments_inter, train_dataset.x.shape[0])
# # with open("./dataset.pickle", 'wb') as of:
# #     pickle.dump(train_dataset, of)
# gc.collect()

data_dir_train = "/mounts/data/proj/ayyoob/align_induction/dataset/dataset_helfi_train_community_word"
data_dir_blinker = "/mounts/data/proj/ayyoob/align_induction/dataset/pruned_alignments_blinker_inter/"
data_dir_grc = "/mounts/data/proj/ayyoob/align_induction/dataset/dataset_helfi_grc_community_word/"
data_dir_heb = "/mounts/data/proj/ayyoob/align_induction/dataset/dataset_helfi_heb_community_word/"

train_dataset = torch.load(f"{data_dir_train}/train_dataset_nox_noedge.torch.bin")
blinker_test_dataset = torch.load(f"{data_dir_blinker}/train_dataset_nox_noedge.torch.bin")
grc_test_dataset = torch.load(f"{data_dir_grc}/train_dataset_nox_noedge.torch.bin")
heb_test_dataset = torch.load(f"{data_dir_heb}/train_dataset_nox_noedge.torch.bin")


In [14]:
import codecs
import collections

postag_map = {"ADJ": 0, "ADP": 1, "ADV": 2, "AUX": 3, "CCONJ": 4, "DET": 5, "INTJ": 6, "NOUN": 7, "NUM": 8, "PART": 9, "PRON": 10, "PROPN": 11, "PUNCT": 12, "SCONJ": 13, "SYM": 14, "VERB": 15, "X": 16}

pos_lang_list = ["eng-x-bible-mixed", "deu-x-bible-newworld", "ces-x-bible-newworld", 
		"fra-x-bible-louissegond","hin-x-bible-newworld", "ita-x-bible-2009", 
		"prs-x-bible-goodnews", "ron-x-bible-2006", "spa-x-bible-newworld"]

def get_db_nodecount(dataset):
	res = 0
	for lang in dataset.nodes_map.values():
		for verse in lang.values():
			res += len(verse)
	
	return res

def get_pos_tags(dataset, pos_lang_list):
	all_tags = {}
	for lang in pos_lang_list:
		if lang not in dataset.nodes_map:
			continue
		all_tags[lang] = {}
		with codecs.open(F"/mounts/work/mjalili/projects/gnn-align/data/pbc_pos_tags/{lang}.conllu", "r", "utf-8") as lang_pos:
			tag_sent = []
			sent_id = ""
			for sline in lang_pos:
				sline = sline.strip()
				if sline == "":
					if sent_id not in dataset.nodes_map[lang]:
						tag_sent = []
						sent_id = ""
						continue

					all_tags[lang][sent_id] = [p[3] for p in tag_sent]
					tag_sent = []
					sent_id = ""
				elif "# verse_id" in sline:
					sent_id = sline.split()[-1]
				elif sline[0] == "#":
					continue
				else:
					tag_sent.append(sline.split("\t"))

	node_count = get_db_nodecount(dataset)
	pos_labels = torch.zeros(node_count, len(postag_map))
	pos_node_cover = collections.defaultdict(list)

	for lang in all_tags:
		for sent_id in all_tags[lang]:
			sent_tags = all_tags[lang][sent_id]
			for w_i in range(len(sent_tags)):
				if w_i not in dataset.nodes_map[lang][sent_id]:
					continue
				pos_labels[dataset.nodes_map[lang][sent_id][w_i], postag_map[sent_tags[w_i]]] = 1
				pos_node_cover[sent_id].append(dataset.nodes_map[lang][sent_id][w_i])

	return pos_labels, pos_node_cover
	#pos_pickle = {"pos_labels": pos_labels, "node_ids_train": pos_ids_train, "node_ids_dev": pos_ids_dev}
	#torch.save(pos_pickle, '/mounts/work/ayyoob/models/gnn/postag')


In [15]:
# blinker_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_blinker_full_community_word.pickle", map_location=torch.device('cpu'))
editf12 = "eng-x-bible-mixed"
editf22 = 'fra-x-bible-louissegond'

test_gold_eng_fra = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/eng-fra.gold"

pros_blinker, surs_blinker = autils.load_gold(test_gold_eng_fra)

blinker_verse_alignments_inter = {}
#blinker_verse_alignments_gdfa = {}
#args = []
#for i,verse in enumerate(blinker_verses):
#    args.append((verse, current_editions))

#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(blinker_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]

#    blinker_verse_alignments_inter[verse] = verse_aligns_inter
#    blinker_verse_alignments_gdfa[verse] = verse_aligns_gdfa

utils.LOG.info("done reading alignments")
#torch.save(blinker_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#torch.save(blinker_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#blinker_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#blinker_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

verses_map = {}

for edit in blinker_test_dataset.nodes_map:
    for verse in blinker_test_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in blinker_test_dataset.nodes_map[edit][verse]:
                verses_map[verse] = blinker_test_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
blinker_verses = [item[0] for item in sorted_verses]


2021-11-03 14:34:45,779 - analytics - INFO - done reading alignments
2021-11-03 14:34:45,780 - analytics - INFO - done saving pruned alignments


In [16]:
#importlib.reload(afeatures)
#grc_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_grc_test_community_word.pickle", map_location=torch.device('cpu'))
editf_fin = "fin-x-bible-helfi"
editf_grc = 'grc-x-bible-helfi'

test_gold_grc = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-grc-gold-alignments_test.txt"

pros_grc, surs_grc = autils.load_gold(test_gold_grc)

grc_test_verse_alignments_inter = {}
grc_test_verse_alignments_gdfa = {}
gc.collect()
#args = []
#for i,verse in enumerate(grc_verses):
#    args.append((verse, current_editions))

#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(grc_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]

#    grc_test_verse_alignments_inter[verse] = verse_aligns_inter
#    grc_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

utils.LOG.info("done reading alignments")
#torch.save(grc_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
#torch.save(grc_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
#grc_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
#grc_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
gc.collect()
print('done reading inter verse alignments')

verses_map = {}

for edit in grc_test_dataset.nodes_map:
    for verse in grc_test_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in grc_test_dataset.nodes_map[edit][verse]:
                verses_map[verse] = grc_test_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
grc_test_verses = [item[0] for item in sorted_verses]

gc.collect()

2021-11-03 14:34:46,144 - analytics - INFO - done reading alignments
2021-11-03 14:34:46,146 - analytics - INFO - done saving pruned alignments


reading inter verse alignments
done reading inter verse alignments


0

In [17]:
#heb_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_heb_test_community_word.pickle", map_location=torch.device('cpu'))

test_gold_heb = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-heb-gold-alignments_test.txt"

pros_heb, surs_heb = autils.load_gold(test_gold_heb)

heb_test_verse_alignments_inter = {}
heb_test_verse_alignments_gdfa = {}
#args = []
#for i,verse in enumerate(heb_verses):
#    args.append((verse, current_editions))

#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(heb_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]

#    heb_test_verse_alignments_inter[verse] = verse_aligns_inter
#    heb_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
#torch.save(heb_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
#torch.save(heb_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
#heb_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
#heb_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
gc.collect()
print('done reading inter verse alignments')

verses_map = {}

for edit in heb_test_dataset.nodes_map:
    for verse in heb_test_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in heb_test_dataset.nodes_map[edit][verse]:
                verses_map[verse] = heb_test_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
heb_test_verses = [item[0] for item in sorted_verses]
gc.collect()

reading inter verse alignments
done reading inter verse alignments


0

In [18]:
verses_map = {}

for edit in train_dataset.nodes_map:
    for verse in train_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in train_dataset.nodes_map[edit][verse]:
                verses_map[verse] = train_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
all_verses = [item[0] for item in sorted_verses]

long_verses = set()

for edit in train_dataset.nodes_map.keys():
    for verse in train_dataset.nodes_map[edit]:
        to_print = False
        for tok in train_dataset.nodes_map[edit][verse]:
            if tok > 150:
                to_print = True
        if to_print == True:
            long_verses.add(verse)


train_verses = all_verses[:]

masked_verses = list(long_verses)
#masked_verses.extend(blinker_verses)

In [19]:
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import random

class POSTAGGNNDataset(Dataset):

    def __init__(self, dataset, verses, edit_files, alignments, node_cover, pos_labels, data_dir, create_data=False, group_size = 20):

        self.pos_labels = pos_labels
        self.data_dir = data_dir
        self.items = self.calculate_size(verses, group_size, node_cover)

        if create_data:
            self.calculate_verse_stats(verses, edit_files, alignments, dataset, data_dir)            
        
    def calculate_size(self, verses, group_size, covered_verses):
        res = []
        for verse in verses:
            covered_nodes = covered_verses[verse]
            random.shuffle(covered_nodes)
            items = [covered_nodes[i:i + group_size] for i in range(0, len(covered_nodes), group_size)]
            res.extend([(verse, i) for i in items])

        return res

    def calculate_verse_stats(self,verses, edition_files, alignments, dataset, data_dir):

        min_edge = 0
        for verse in tqdm(verses):
            min_nodes = 99999999999999
            max_nodes = 0
            edges_tmp = [[],[]]
            x_tmp = []
            features = []
            for i,editf1 in enumerate(edition_files):
                for j,editf2 in enumerate(edition_files[i+1:]):
                    aligns = autils.get_aligns(editf1, editf2, alignments[verse])
                    if aligns != None:
                        for align in aligns:
                            try:
                                n1,_ = gutils.node_nom(verse, editf1, align[0], None, dataset.nodes_map, x_tmp, edition_files, features)
                                n2,_ = gutils.node_nom(verse, editf2, align[1], None, dataset.nodes_map, x_tmp, edition_files, features)
                                edges_tmp[0].extend([n1, n2])

                                max_nodes = max(n1, n2, max_nodes)
                                min_nodes = min(n1, n2, min_nodes)
                            except Exception as e:
                                print(editf1, editf2, verse)
                                raise(e)

            self.verse_info = {}

            self.verse_info['padding'] = min_nodes
            
            self.verse_info['x'] = torch.clone(dataset.x[min_nodes:max_nodes+1,:])
            
            self.verse_info['edge_index'] = torch.clone(dataset.edge_index[:, min_edge : min_edge + len(edges_tmp[0])] - min_nodes)

            if torch.min(self.verse_info['edge_index']) != 0:
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info['edge_index']))
            
            if self.verse_info['x'].shape[0] != torch.max(self.verse_info['edge_index']) + 1 :
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info['edge_index']))
            
            min_edge = min_edge + len(edges_tmp[0])

            torch.save(self.verse_info, f"{data_dir}/verses/{verse}_info.torch.bin")
        
        dataset.x = None
        dataset.edge_index = None
        torch.save(dataset, f"{data_dir}/train_dataset_nox_noedge.torch.bin")
    
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        verse, nodes = self.items[idx]
        
        self.verse_info = {verse: torch.load(f'{self.data_dir}/verses/{verse}_info.torch.bin')}

        word_number = self.verse_info[verse]['x'][:, 9]
        word_number = torch.unsqueeze(word_number, 1)
        self.verse_info[verse]['x'] = torch.cat((self.verse_info[verse]['x'], word_number), dim=1)
        
        return {'verse':verse, 'x':self.verse_info[verse]['x'], 'edge_index':self.verse_info[verse]['edge_index'], 
                'pos_classes': self.pos_labels[nodes, :], 'pos_index': torch.LongTensor(nodes) - self.verse_info[verse]['padding']}


# train_pos_labels, train_pos_node_cover = get_pos_tags(train_dataset, pos_lang_list)
# torch.save({'pos_labels':train_pos_labels, 'pos_node_cover':train_pos_node_cover}, f'{data_dir_train}/pos_data.torch.bin')
pos_data = torch.load(f'{data_dir_train}/pos_data.torch.bin')
train_pos_labels, train_pos_node_cover = pos_data['pos_labels'], pos_data['pos_node_cover']
gnn_dataset_train_pos = POSTAGGNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter,
                       train_pos_node_cover, train_pos_labels, data_dir_train, group_size = 100)

# blinker_pos_labels, blinker_pos_node_cover = get_pos_tags(blinker_test_dataset, pos_lang_list)
# torch.save({'pos_labels':blinker_pos_labels, 'pos_node_cover': blinker_pos_node_cover}, f'{data_dir_blinker}/pos_data.torch.bin')
pos_data = torch.load(f'{data_dir_blinker}/pos_data.torch.bin')
blinker_pos_labels, blinker_pos_node_cover = pos_data['pos_labels'], pos_data['pos_node_cover']
gnn_dataset_blinker_pos = POSTAGGNNDataset(blinker_test_dataset, blinker_verses, current_editions, blinker_verse_alignments_inter,
                             blinker_pos_node_cover, blinker_pos_labels, data_dir_blinker, group_size = 100)

#grc_pos_labels, grc_pos_node_cover = get_pos_tags(grc_test_dataset)
#torch.save({'pos_labels':grc_pos_labels, 'pos_node_cover': grc_pos_node_cover}, f'{data_dir_grc}/pos_data.torch.bin')
pos_data = torch.load(f'{data_dir_grc}/pos_data.torch.bin')
grc_pos_labels, grc_pos_node_cover = pos_data['pos_labels'], pos_data['pos_node_cover']
gnn_dataset_grc_pos = POSTAGGNNDataset(grc_test_dataset, grc_test_verses, current_editions, grc_test_verse_alignments_inter,
                            grc_pos_node_cover, grc_pos_labels, data_dir_grc, group_size = 10000)

#heb_pos_labels, heb_pos_node_cover = get_pos_tags(heb_test_dataset)
#torch.save({'pos_labels':heb_pos_labels, 'pos_node_cover': heb_pos_node_cover}, f'{data_dir_heb}/pos_data.torch.bin')
pos_data = torch.load(f'{data_dir_heb}/pos_data.torch.bin')
heb_pos_labels, heb_pos_node_cover = pos_data['pos_labels'], pos_data['pos_node_cover']
gnn_dataset_heb_pos = POSTAGGNNDataset(heb_test_dataset, heb_test_verses, current_editions, heb_test_verse_alignments_inter,
                            heb_pos_node_cover, heb_pos_labels, data_dir_heb, group_size = 10000)

print(len(gnn_dataset_train_pos))
gc.collect()


57706


0

In [20]:
no_eng_langs = pos_lang_list[:]
no_eng_langs.remove('eng-x-bible-mixed')

train_pos_labels, train_pos_node_cover = get_pos_tags(train_dataset, no_eng_langs)
gnn_dataset_train_pos = POSTAGGNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter,
                       train_pos_node_cover, train_pos_labels, data_dir_train, group_size = 100)

blinker_pos_labels, blinker_pos_node_cover = get_pos_tags(blinker_test_dataset, ['eng-x-bible-mixed'])
gnn_dataset_blinker_pos = POSTAGGNNDataset(blinker_test_dataset, blinker_verses, current_editions, blinker_verse_alignments_inter,
                             blinker_pos_node_cover, blinker_pos_labels, data_dir_blinker, group_size = 100)


In [21]:
def save_model(model, name):
    model.encoder.feature_encoder.feature_types[0] = afeatures.OneHotFeature(20, 83, 'editf')
    model.encoder.feature_encoder.feature_types[1] = afeatures.OneHotFeature(32, 150, 'position')
    model.encoder.feature_encoder.feature_types[2] = afeatures.FloatFeature(4, 'degree_centrality')
    model.encoder.feature_encoder.feature_types[3] = afeatures.FloatFeature(4, 'closeness_centrality')
    model.encoder.feature_encoder.feature_types[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
    model.encoder.feature_encoder.feature_types[5] = afeatures.FloatFeature(4, 'load_centrality')
    model.encoder.feature_encoder.feature_types[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
    model.encoder.feature_encoder.feature_types[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
    model.encoder.feature_encoder.feature_types[8] = afeatures.OneHotFeature(32, 250, 'community_2')
    model.encoder.feature_encoder.feature_types[9] = afeatures.MappingFeature(100, 'word')
    torch.save(model, f'/mounts/work/ayyoob/models/gnn/checkpoint/postagging/pos_tagging_{name}_' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.pickle')

In [22]:
#test(1, test_data_loader) 

#finetune_pos_labels, finetune_pos_node_cover = get_pos_tags(train_dataset, ['eng-x-bible-mixed'])
#gnn_dataset_finetune_pos = POSTAGGNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter,
#                       finetune_pos_node_cover, finetune_pos_labels, data_dir_train, group_size = 100)
#finetune_data_loader = DataLoader(gnn_dataset_finetune_pos, batch_size=1, shuffle=False)

# train(1, finetune_data_loader, max_batches=1000)
# test(1, test_data_loader) 

In [23]:
from tqdm import tqdm

mask_language = True

torch.cuda.set_device(0)
features = train_dataset.features

train_data_loader = DataLoader(gnn_dataset_train_pos, batch_size=1, shuffle=True)
test_data_loader = DataLoader(gnn_dataset_blinker_pos, batch_size=1, shuffle=True)

clean_memory()
drop_out = 0
n_head = 1

channels = 512

decoder_in_dim = n_head * channels 
decoder = POSDecoder(decoder_in_dim, decoder_in_dim*2, len(postag_map))
model = torch.load('/mounts/work/ayyoob/models/gnn/checkpoint/gnn_512_flggll_word_halfTrain_nofeatlinear_encoderlineear_decoderonelayer20210910-235352-.pickle')
freeze_encoders_embedding(model.encoder.feature_encoder)

model.decoder = decoder
model.to(dev)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

torch.set_printoptions(edgeitems=5)
print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

for epoch in range(1, 2):
   print(f"\n----------------epoch {epoch} ---------------")
    
   #if epoch % 1 == 0:
   #    train_neg_edge_index = gutils.get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
       #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

   train(epoch, train_data_loader)
#   save_model(model, 'freeze-embedding_noLang')
   test(epoch, test_data_loader) 
   clean_memory()

1/0

  0%|          | 0/50092 [00:00<?, ?it/s]

doing it
model params - decoder params - conv1 237075569 542737

----------------epoch 1 ---------------


  1%|          | 497/50092 [00:44<45:41, 18.09it/s]

loss: 48799.44767956436
testing 1


  1%|          | 502/50092 [01:01<20:11:44,  1.47s/it]

...

loss: 32640.671494985
testing 1


 42%|████▏     | 21003/50092 [20:57<1:43:03,  4.70it/s]

test, epoch: 1, ACC: 0.7486960010699478


 43%|████▎     | 21336/50092 [21:11<28:33, 16.78it/s]


KeyboardInterrupt: 

In [ ]:
#i = sag
#batch = khar
#verse = gav
#print(i, verse)

#keys = list(gnn_dataset.verse_info.keys())

#gnn_dataset.verse_info[verse]


In [ ]:
model = torch.load('/mounts/work/ayyoob/models/gnn/checkpoint/pos_tagging_zeta_1_20210915-011448.pickle')
torch.cuda.set_device(4)
model.to(dev)
batch = 0

In [86]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load("/mounts/work/ayyoob/models/w2v/word2vec_helfi_langs_15e.model")

print(w2v_model.wv.vectors.shape)
word_vectors = torch.from_numpy(w2v_model.wv.vectors).float()

(2354770, 100)


In [182]:
def get_words_tag_frequence(model, word_count, class_count, data_loader, tag_frequencies=None, from_gold_data=False):
    
    res = tag_frequencies
    if res == None:
        res = torch.ones(word_count, class_count)
        res[:, :] = 0.0000001
    
    
    data_encoder = DataEncoder(data_loader, model)

    with torch.no_grad():

        for z, verse, i, batch in data_encoder:
            index = batch['pos_index'][0].to(dev)
            
            if from_gold_data:
                tags_onehot = batch['pos_classes'][0]
            else:
                tags_onehot = model.decoder(z, index)

            max_values, pos_tags = torch.max(torch.softmax(tags_onehot, dim=1), 1)
            word_pos = batch['x'][0][index, 9]

            if not from_gold_data:
                accepted_values = max_values > 0.5
                word_pos = word_pos[accepted_values]
                pos_tags = pos_tags[accepted_values]

            res[word_pos.long(), pos_tags.long()] += 1
    
    sm = torch.sum(res, dim=1)
    res_normalized = (res.transpose(1,0) / sm).transpose(1,0)
    
    return res_normalized, res

#gnn_dataset_train_pos_bigbatch = POSTAGGNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter,
#                    train_pos_node_cover, train_pos_labels, data_dir_train, group_size = 10000)
#train_data_loader_bigbatch = DataLoader(gnn_dataset_train_pos_bigbatch, batch_size=1, shuffle=True)
normalized_tag_frequencies, tag_frequencies_other = get_words_tag_frequence(model, 2354770, len(postag_map), train_data_loader_bigbatch, from_gold_data=True)

#english_train_pos_labels, english_train_pos_node_cover = get_pos_tags(train_dataset,['eng-x-bible-mixed'])
#gnn_dataset_english_pos = POSTAGGNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter,
#                    english_train_pos_node_cover, english_train_pos_labels, data_dir_train, group_size = 500)
#english_data_loader = DataLoader(gnn_dataset_english_pos, batch_size=1, shuffle=False)

#normalized_tag_frequencies_english, tag_frequencies_english = get_words_tag_frequence(model, 2354770, len(postag_map), english_data_loader)

tag_frequencies = tag_frequencies_other + tag_frequencies_english
sm = torch.sum(tag_frequencies, dim=1)
normalized_tag_frequencies = (tag_frequencies.transpose(1,0) / sm).transpose(1,0)

100%|██████████| 24078/24078 [08:30<00:00, 47.16it/s]


In [186]:
word_frequencies = torch.sum(tag_frequencies_english, dim=1)
tag_frequencies_copy = tag_frequencies.detach().clone()

tag_frequencies_copy[torch.logical_and(word_frequencies>0.1, word_frequencies<3), :] = 0.0000001

uniform_noise = torch.BoolTensor(tag_frequencies.size(0))
uniform_noise[:] = True
shuffle_tensor = torch.randperm(tag_frequencies.size(0))[:int(tag_frequencies.size(0)*0.7)]
uniform_noise[shuffle_tensor] = False
tag_frequencies_copy[torch.logical_and(uniform_noise, word_frequencies < 0.1), :] = 0.0000001

sm = torch.sum(tag_frequencies_copy, dim=1)
normalized_tag_frequencies = (tag_frequencies_copy.transpose(1,0) / sm).transpose(1,0)

In [181]:

#normalized_gold_frequencies, gold_frequencies_all = get_words_tag_frequence(model, 2354770, len(postag_map), english_data_loader, from_gold_data=True)

#gold_frequencies_all = gold_frequencies
word_frequencies = torch.sum(gold_frequencies_all, dim=1)

subjectword_indices =  word_frequencies > 0.1
print(word_frequencies.shape)
gold_frequencies = gold_frequencies_all[subjectword_indices, :]
predicted_frequencies = tag_frequencies_english[subjectword_indices, :]
wordtype_frequencies = word_frequencies[subjectword_indices]
print(gold_frequencies.shape)

_, gold_tags = torch.max(gold_frequencies, dim=1)
_, predicted_tags = torch.max(predicted_frequencies, dim=1)

sorted_wordtype_frequencies, sort_pattern = torch.sort(wordtype_frequencies, descending=True)

sorted_gold_tags = gold_tags[sort_pattern]
sorted_predicted_tags = predicted_tags[sort_pattern]
quarter_size = int(sorted_gold_tags.size(0)/2.0)

print('quarter size', quarter_size)
print("general accuracy", torch.sum(gold_tags == predicted_tags)/predicted_tags.size(0))
print('first quarter accuracy', torch.sum(sorted_gold_tags[:quarter_size] == sorted_predicted_tags[:quarter_size])/quarter_size)
print('last part accuracy', torch.sum(sorted_gold_tags[1*quarter_size:] == sorted_predicted_tags[1*quarter_size:])/sorted_predicted_tags[1*quarter_size:].size(0))

print('total token count', torch.sum(wordtype_frequencies))
print('first quarter words token count', torch.sum(word_frequencies[:quarter_size]))

print('1st frequency', sorted_wordtype_frequencies[0])
print('10st frequency', sorted_wordtype_frequencies[10])
print('100st frequency', sorted_wordtype_frequencies[100])
print('100st frequency', sorted_wordtype_frequencies[736])
print('1000st frequency', sorted_wordtype_frequencies[1000])
print('10000st frequency', sorted_wordtype_frequencies[10000])


torch.Size([2354770])
torch.Size([14734, 17])
quarter size 7367
general accuracy tensor(0.6985)
first quarter accuracy tensor(0.7789)
last part accuracy tensor(0.6182)
total token count tensor(550089.)
first quarter words token count tensor(280803.)
1st frequency tensor(21465.)
10st frequency tensor(5141.)
100st frequency tensor(796.)
100st frequency tensor(79.)
1000st frequency tensor(51.)
10000st frequency tensor(1.0000)


In [89]:
features.pop()
#features.append(afeatures.MappingFeature(len(postag_map), 'tag_priors'))


In [170]:
#normalized_tag_frequencies = torch.softmax(tag_frequencies_copy, dim=1)

sm = torch.sum(tag_frequencies_copy, dim=1)
normalized_tag_frequencies = (tag_frequencies_copy.transpose(1,0) / sm).transpose(1,0)

In [187]:
train_data_loader = DataLoader(gnn_dataset_train_pos, batch_size=1, shuffle=True)
test_data_loader = DataLoader(gnn_dataset_blinker_pos, batch_size=1, shuffle=True)

#features.append(afeatures.MappingFeature(len(postag_map), 'tag_priors'))
channels = 512
in_dim = sum(t.out_dim for t in features)
##model.encoder.feature_encoder.feature_types.append(afeatures.MappingFeature(len(postag_map), 'tag_priors'))
##model.encoder.feature_encoder.layers.append(afeatures.MappingEncoding(normalized_tag_frequencies, freeze=True))
##model.encoder.conv1 = pyg_nn.GATConv(in_dim, 2*channels, heads= n_head)

decoder_in_dim = n_head * channels 
decoder = POSDecoder(decoder_in_dim, decoder_in_dim*2, len(postag_map))
model = pyg_nn.GAE(Encoder(in_dim, channels, features, n_head, edge_feature_dim=len([])), decoder).to(dev)
freeze_encoders_embedding(model.encoder.feature_encoder)

model.to(dev)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

for epoch in range(1, 2):
   print(f"\n----------------epoch {epoch} ---------------")
    
   #if epoch % 1 == 0:
   #    train_neg_edge_index = gutils.get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
       #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

   train(epoch, train_data_loader)
#   save_model(model, 'freeze-embedding_noLang')
   test(epoch, test_data_loader) 
   clean_memory()

  0%|          | 3/50092 [00:00<36:34, 22.82it/s]

doing it
doing it
model params - decoder params - conv1 277124067 542737

----------------epoch 1 ---------------


  1%|          | 497/50092 [00:19<29:48, 27.73it/s]

loss: 50028.33756018989
testing 1


  1%|          | 503/50092 [00:24<4:33:43,  3.02it/s]

...

loss: 24343.04919190705
testing 1


 76%|███████▌  | 38003/50092 [29:04<37:11,  5.42it/s]

test, epoch: 1, total:7477 ACC: 0.7810619232312425


 76%|███████▌  | 38144/50092 [29:08<09:07, 21.81it/s]


KeyboardInterrupt: 

In [ ]:
def get_yoruba_postags(dataset, gnn_dataset, data_loader):
    edit ='yor-x-bible-2010'

    model.eval()
    res = {}
    data_endoer = DataEncoder(data_loader, model)
    
    with torch.no_grad():

        for z, verse, _, _ in data_endoer:
            if verse in dataset.nodes_map[edit]:

                index = []
                toks = list(dataset.nodes_map[edit][verse].keys())
                for i in toks:
                    index.append(dataset.nodes_map[edit][verse][i])
                index = torch.LongTensor(index).to(dev) - gnn_dataset.verse_info[verse]['padding']

                preds = model.decoder(z, index)

                _, predicted = torch.max(preds, 1)

                res[verse] = {toks[i]:predicted[i].item() for i in range(len(toks))}

    return res

yoruba_pos_tags = {}
res_ = get_yoruba_postags(train_dataset, gnn_dataset_train_pos, DataLoader(gnn_dataset_train_pos, batch_size=1, shuffle=False))
yoruba_pos_tags.update(res_)

res_ = get_yoruba_postags(heb_test_dataset, gnn_dataset_heb_pos, DataLoader(gnn_dataset_heb_pos, batch_size=1, shuffle=False))
yoruba_pos_tags.update(res_)

res_ = get_yoruba_postags(grc_test_dataset, gnn_dataset_grc_pos, DataLoader(gnn_dataset_grc_pos, batch_size=1, shuffle=False))
yoruba_pos_tags.update(res_)

res_ = get_yoruba_postags(blinker_test_dataset, gnn_dataset_blinker_pos, DataLoader(gnn_dataset_blinker_pos, batch_size=1, shuffle=False))
yoruba_pos_tags.update(res_)

torch.save(yoruba_pos_tags, '/mounts/work/ayyoob/results/gnn_align/yoruba/pos_tags_freeze-embedding_noLang.pickle')

In [ ]:
len(yoruba_pos_tags.keys())


In [ ]:
global model, decoder
#1/0

decoder = None
model = None

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:

features = blinker_test_dataset.features[:]
#features_edge = train_dataset.features_edge[:]
from pprint import pprint
#print('indim',in_dim)
#features[-1].out_dim = 50
for i in features:
    #if i.type==3:
    #    i.out_dim=4
    print(vars(i))

#sum(p.out_dim for p in features)
#train_dataset.features.pop()
#train_dataset.features[0] = afeatures.OneHotFeature(20, 83, 'editf')
#train_dataset.features[1] = afeatures.OneHotFeature(32, 150, 'position')
#train_dataset.features[2] = afeatures.FloatFeature(4, 'degree_centrality')
#train_dataset.features[3] = afeatures.FloatFeature(4, 'closeness_centrality')
#train_dataset.features[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
#train_dataset.features[5] = afeatures.FloatFeature(4, 'load_centrality')
#train_dataset.features[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
#train_dataset.features[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
##train_dataset.features.append(afeatures.MappingFeature(100, 'word'))
#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word.pickle")
#torch.save(train_dataset.features[-3], "./features.tmp")

{'type': 1, 'out_dim': 20, 'global_normalize': False, 'name': 'edit_f', 'Active': True, 'n_classes': 83}
{'type': 1, 'out_dim': 32, 'global_normalize': False, 'name': 'position', 'Active': True, 'n_classes': 150}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': '{centrality_type}', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': '{centrality_type}', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': '{centrality_type}', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': '{centrality_type}', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': '{centrality_type}', 'Active': True}
{'type': 1, 'out_dim': 32, 'global_normalize': False, 'name': '{name}', 'Active': True, 'n_classes': 200}
{'type': 1, 'out_dim': 32, 'global_normalize': False, 'name': '{name}', 'Active': True, 'n_classes': 200}
{'type': 6, 'out_dim': 100, 'global_normalize': False, 'name': 'word', 'Active': True

In [ ]:
nodes_map = train_dataset.nodes_map
bad_edition_files = []
for edit in nodes_map:
    bad_count = 0
    for verse in nodes_map[edit]:
        if len(nodes_map[edit][verse].keys()) < 2:
            bad_count += 1
        if bad_count > 1:
            bad_edition_files.append(edit)
            break
print(bad_edition_files)

In [ ]:
all_japanese_nodes = set()
nodes_map = train_dataset.nodes_map

for bad_editionf in bad_edition_files:
    for verse in nodes_map[bad_editionf]:
        for item in nodes_map[bad_editionf][verse].items():
            all_japanese_nodes.add(item[1])

print(" all japansese nodes: ", len(all_japanese_nodes))
edge_index = train_dataset.edge_index.to('cpu')
remaining_edges_index = []
for i in tqdm(range(0, edge_index.shape[1], 2)):
    if edge_index[0, i].item() not in all_japanese_nodes and edge_index[0, i+1].item() not in all_japanese_nodes:
        remaining_edges_index.extend([i, i+1])

print('original total edges count', edge_index.shape)
print('remaining edge count', len(remaining_edges_index))
train_dataset.edge_index = edge_index[:, remaining_edges_index]
train_dataset.edge_index.shape
